In [24]:
import geopandas as gpd
from rasterio.features import rasterize
from rasterio import transform

In [10]:
# load landcover data
lc = gpd.read_file("data/CBS_Land_use_Netherlands_2017/CBS_Land_use_Netherlands_2017.gpkg")
lc

c:\Users\ramiq\Application\hack4good\.venv\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'CBS_Land_use_Netherlands_2017.gpkg': 'bestand_bodemgebruik_2017' (default), 'bestand_bodemgebruik_2017_bodemgebruik'. Specify layer parameter to avoid this warning.
  result = read_func(


,bg2017,bodemgebruik,categorie,geometry
0,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((43278.376 374529.115, 43278.37..."
1,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((120111.119 403198.578, 120116...."
2,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((165518.079 403933.207, 165525...."
3,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((99772.811 423051.852, 99772.09..."
4,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((105349.804 429333.505, 105373...."
...,...,...,...,...
171538,82,Water,Westerschelde,"MULTIPOLYGON (((57306.449 386560.951, 57313.32..."
171539,82,Water,Westerschelde,"MULTIPOLYGON (((38510.517 388318.021, 38512.37..."
171540,83,Water,Noordzee,"MULTIPOLYGON (((25339.899 399998.153, 25334.94..."
171541,83,Water,Noordzee,"MULTIPOLYGON (((240000 625000, 240000 620096.1..."


In [11]:
# The land label
labels = lc["categorie"].unique()
labels_string = ";".join(labels)
labels_string

'Spoorterrein;Hoofdweg;Vliegveld;Woongebied;Detailhandel en horeca;Openbare voorziening;Sociaal-culturele voorziening;Bedrijfsterrein;Stortplaats;Wrakkenopslagplaats;Begraafplaats;Delfstofwinplaats;Bouwterrein;Semi-verhard overig terrein;Park en plantsoen;Sportterrein;Volkstuin;Dagrecreatief terrein;Verblijfsrecreatief terrein;Glastuinbouw;Overig agrarisch terrein;Bos;Open droog natuurlijk terrein;Open nat natuurlijk terrein;IJsselmeer & Markermeer;Afgesloten zeearm;Rijn & Maas;Randmeer;Spaarbekken;Water met recreatieve functie;Water met delfstofwinningsfunctie;Vloei- en/of slibveld;Overig binnenwater;Waddenzee, Eems & Dollard;Oosterschelde;Westerschelde;Noordzee;Buitenland'

In [12]:
# Label translate
labels_en = 'Railway site;Main road;Airport;Residential area;Retail and catering;Public facility;Socio-cultural facility;Industrial estate;Landfill;Wreck storage site;Cemetery;Mineral extraction site;Construction site;Semi-paved other terrain;Park and public garden;Sports field;Allotment garden;Day recreation area;Recreational accommodation area;Greenhouse horticulture;Other agricultural terrain;Forest;Open dry natural terrain;Open wet natural terrain;IJsselmeer & Markermeer;Closed sea arm;Rhine & Meuse;Randmeer;Reservoir;Water with recreational function;Water with mineral extraction function;Fluid and/or silt field;Other inland water;Wadden Sea, Eems & Dollard;Oosterschelde;Westerschelde;North Sea;Abroad'
labels_en = labels_en.split(";")
labels_en

['Railway site',
 'Main road',
 'Airport',
 'Residential area',
 'Retail and catering',
 'Public facility',
 'Socio-cultural facility',
 'Industrial estate',
 'Landfill',
 'Wreck storage site',
 'Cemetery',
 'Mineral extraction site',
 'Construction site',
 'Semi-paved other terrain',
 'Park and public garden',
 'Sports field',
 'Allotment garden',
 'Day recreation area',
 'Recreational accommodation area',
 'Greenhouse horticulture',
 'Other agricultural terrain',
 'Forest',
 'Open dry natural terrain',
 'Open wet natural terrain',
 'IJsselmeer & Markermeer',
 'Closed sea arm',
 'Rhine & Meuse',
 'Randmeer',
 'Reservoir',
 'Water with recreational function',
 'Water with mineral extraction function',
 'Fluid and/or silt field',
 'Other inland water',
 'Wadden Sea, Eems & Dollard',
 'Oosterschelde',
 'Westerschelde',
 'North Sea',
 'Abroad']

In [13]:
# Remap dict from dutch to en
remap_lc = {}
for x in range(len(labels)):
  remap_lc[labels[x]] = labels_en[x]
remap_lc

{'Spoorterrein': 'Railway site',
 'Hoofdweg': 'Main road',
 'Vliegveld': 'Airport',
 'Woongebied': 'Residential area',
 'Detailhandel en horeca': 'Retail and catering',
 'Openbare voorziening': 'Public facility',
 'Sociaal-culturele voorziening': 'Socio-cultural facility',
 'Bedrijfsterrein': 'Industrial estate',
 'Stortplaats': 'Landfill',
 'Wrakkenopslagplaats': 'Wreck storage site',
 'Begraafplaats': 'Cemetery',
 'Delfstofwinplaats': 'Mineral extraction site',
 'Bouwterrein': 'Construction site',
 'Semi-verhard overig terrein': 'Semi-paved other terrain',
 'Park en plantsoen': 'Park and public garden',
 'Sportterrein': 'Sports field',
 'Volkstuin': 'Allotment garden',
 'Dagrecreatief terrein': 'Day recreation area',
 'Verblijfsrecreatief terrein': 'Recreational accommodation area',
 'Glastuinbouw': 'Greenhouse horticulture',
 'Overig agrarisch terrein': 'Other agricultural terrain',
 'Bos': 'Forest',
 'Open droog natuurlijk terrein': 'Open dry natural terrain',
 'Open nat natuurlijk

In [14]:
# Add land cover en to english
lc["label_en"] = lc["categorie"]
lc = lc.replace({"label_en": remap_lc})
lc

,bg2017,bodemgebruik,categorie,geometry,label_en
0,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((43278.376 374529.115, 43278.37...",Railway site
1,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((120111.119 403198.578, 120116....",Railway site
2,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((165518.079 403933.207, 165525....",Railway site
3,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((99772.811 423051.852, 99772.09...",Railway site
4,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((105349.804 429333.505, 105373....",Railway site
...,...,...,...,...,...
171538,82,Water,Westerschelde,"MULTIPOLYGON (((57306.449 386560.951, 57313.32...",Westerschelde
171539,82,Water,Westerschelde,"MULTIPOLYGON (((38510.517 388318.021, 38512.37...",Westerschelde
171540,83,Water,Noordzee,"MULTIPOLYGON (((25339.899 399998.153, 25334.94...",North Sea
171541,83,Water,Noordzee,"MULTIPOLYGON (((240000 625000, 240000 620096.1...",North Sea


In [15]:
# save the new lc
lc.to_file("output/lc_nl_en.shp")

C:\Users\ramiq\AppData\Local\Temp\ipykernel_34408\2258989107.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lc.to_file("output/lc_nl_en.shp")
c:\Users\ramiq\Application\hack4good\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'bodemgebruik' to 'bodemgebru'
  ogr_write(


In [16]:
# Sort lc by class value
lc = lc.sort_values("bg2017")
class_id = lc["bg2017"].unique()
class_id

array([10, 11, 12, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 35, 40, 41, 42,
       43, 44, 50, 51, 60, 61, 62, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80,
       81, 82, 83, 90], dtype=int32)

In [17]:
# Put the score in the lc label
values = "1	0	0	2	0	1	1	0	1	0	2	1	0	2	5	4	6	5	4	3	4	9	8	9	7	7	6	6	5	5	4	5	6	9	8	8	7	0"
values = values.split("\t")
values = list(map(lambda x: int(x), values))
values

# Remap values
remap_score = {}
for x in range(len(class_id)):
  remap_score[class_id[x]] = values[x]

lc["score"] = lc["bg2017"]
lc = lc.replace({"score": remap_score})
lc

,bg2017,bodemgebruik,categorie,geometry,label_en,score
0,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((43278.376 374529.115, 43278.37...",Railway site,1
1,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((120111.119 403198.578, 120116....",Railway site,1
2,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((165518.079 403933.207, 165525....",Railway site,1
3,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((99772.811 423051.852, 99772.09...",Railway site,1
4,10,Spoorterrein,Spoorterrein,"MULTIPOLYGON (((105349.804 429333.505, 105373....",Railway site,1
...,...,...,...,...,...,...
171536,82,Water,Westerschelde,"MULTIPOLYGON (((56045.988 378949.159, 55883.21...",Westerschelde,8
171535,82,Water,Westerschelde,"MULTIPOLYGON (((55881.326 377894.135, 55881.29...",Westerschelde,8
171541,83,Water,Noordzee,"MULTIPOLYGON (((240000 625000, 240000 620096.1...",North Sea,7
171540,83,Water,Noordzee,"MULTIPOLYGON (((25339.899 399998.153, 25334.94...",North Sea,7


In [18]:
# Save the lc scored
# save the new lc
lc.to_file("output/lc_nl_en_scored.shp")

C:\Users\ramiq\AppData\Local\Temp\ipykernel_34408\1864366352.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lc.to_file("output/lc_nl_en_scored.shp")
c:\Users\ramiq\Application\hack4good\.venv\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'bodemgebruik' to 'bodemgebru'
  ogr_write(


In [32]:
# Get extent of the vector
extent = lc.total_bounds

# Calculate the the pixels potential
height = extent[3] - extent[1]
width = extent[2] - extent[0]
shape = (int(height / 10), int(width/ 10))
shape

# Transform affine
affine_transform = transform.from_bounds(*extent, shape[1], shape[0])
affine_transform

# CRS
lc_crs = lc.crs

In [ ]:
# Vector to raster
raster = rasterize(
  [data for data in zip(lc.geometry, lc["score"])],
  out_shape=shape,
  transform=affine_transform,
  nodata=-9999,
  fill=-9999,
  dtype="int16",
)
raster